<a href="https://colab.research.google.com/github/nihal-25/ML-LAB/blob/main/1BM22CS178_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import copy

In [2]:
dataset = pd.read_csv('weather.csv')
X = dataset.iloc[:,:].values
X

array([[8.0, 24.3, 0.0, ..., 'No', 3.6, 'Yes'],
       [14.0, 26.9, 3.6, ..., 'Yes', 3.6, 'Yes'],
       [13.7, 23.4, 3.6, ..., 'Yes', 39.8, 'Yes'],
       ...,
       [12.5, 19.9, 0.0, ..., 'No', 0.0, 'No'],
       [12.5, 26.9, 0.0, ..., 'No', 0.0, 'No'],
       [12.3, 30.2, 0.0, ..., 'No', 0.0, 'No']], dtype=object)

In [3]:
attribute = ['Outlook', 'Temp', 'Humidity', 'Wind']


In [4]:
class Node(object):
    def __init__(self):
        self.value = None
        self.decision = None
        self.child = None

In [5]:
def findEntropy(data, rows):
    yes=0
    no=0
    ans=-1
    idx=len(data[0])-1
    entropy=0

    for i in rows:
        if data[i][idx]=='Yes':
            yes=yes+1
        else:
            no=no+1

    x=yes/(yes+no)
    y=no/(yes+no)
    if x!=0 and y!=0:
        entropy= -1*(x*math.log2(x)+y*math.log2(y))
    if x==1:
        ans = 1
    if y==1:
        ans = 0
    return entropy, ans

In [6]:
def findMaxGain(data, rows, columns):
    maxGain = 0
    retidx = -1
    entropy, ans = findEntropy(data, rows)
    if entropy == 0:
        """if ans == 1:
            print("Yes")
        else:
            print("No")"""
        return maxGain, retidx, ans
    for j in columns:
        mydict = {}
        idx = j
        for i in rows:
            key = data[i][idx]
            if key not in mydict:
                mydict[key] = 1
            else:
                mydict[key] = mydict[key] + 1
        gain = entropy

        # print(mydict)
        for key in mydict:
            yes = 0
            no = 0
            for k in rows:
                if data[k][j] == key:
                    if data[k][-1] == 'Yes':
                        yes = yes + 1
                    else:
                        no = no + 1
            # print(yes, no)
            x = yes/(yes+no)
            y = no/(yes+no)
            # print(x, y)
            if x != 0 and y != 0:
                gain += (mydict[key] * (x*math.log2(x) + y*math.log2(y)))/14
        # print(gain)
        if gain > maxGain:
            # print("hello")
            maxGain = gain
            retidx = j

    return maxGain, retidx, ans

In [7]:
def buildTree(data, rows, columns):

    maxGain, idx, ans = findMaxGain(X, rows, columns)
    root = Node()
    root.childs = []
    # print(maxGain)

    if maxGain == 0:
        if ans == 1:
            root.value = 'Yes'
        else:
            root.value = 'No'
        return root

    root.value = attribute[idx]
    mydict = {}
    for i in rows:
        key = data[i][idx]
        if key not in mydict:
            mydict[key] = 1
        else:
            mydict[key] += 1

    newcolumns = copy.deepcopy(columns)
    newcolumns.remove(idx)
    for key in mydict:
        newrows = []
        for i in rows:
            if data[i][idx] == key:
                newrows.append(i)
        # print(newrows)
        temp = buildTree(data, newrows, newcolumns)
        temp.decision = key
        root.childs.append(temp)
    return root

In [8]:
def traverse(root, level=0):
    # Create indentation based on the level
    indent = "    " * level  # 4 spaces per level

    # Print the decision and value of the current node with indentation
    print(f"{indent}├── Decision: {root.decision}, Value: {root.value}")

    # Traverse the children, if they exist
    for i, child in enumerate(root.childs):
        if i == len(root.childs) - 1:  # Last child
            traverse(child, level + 1)
        else:
            traverse(child, level + 1)

In [9]:
def calculate():
    rows = [i for i in range(0, 14)]
    columns = [i for i in range(0, 4)]
    root = buildTree(X, rows, columns)
    root.decision = 'Start'
    traverse(root)

In [10]:
calculate()

├── Decision: Start, Value: Outlook
    ├── Decision: 8.0, Value: Yes
    ├── Decision: 14.0, Value: Yes
    ├── Decision: 13.7, Value: Yes
    ├── Decision: 13.3, Value: Yes
    ├── Decision: 7.6, Value: No
    ├── Decision: 6.2, Value: No
    ├── Decision: 6.1, Value: No
    ├── Decision: 8.3, Value: No
    ├── Decision: 8.8, Value: Yes
    ├── Decision: 8.4, Value: No
    ├── Decision: 9.1, Value: No
    ├── Decision: 8.5, Value: No
    ├── Decision: 10.1, Value: No
    ├── Decision: 12.1, Value: No


In [11]:
from graphviz import Source

dot_code = """
digraph G {
    edge [dir=forward]
    node [shape=box, style=bold]

    A [label="OUTLOOK"]
    B [label="HUMIDITY"]
    C [label="WIND"]

    D [label="NO", shape=plaintext]
    E [label="YES", shape=plaintext]
    F [label="YES", shape=plaintext]
    G [label="NO", shape=plaintext]

    A -> B [label="SUNNY"]
    A -> E [label="OVERCAST"]
    A -> C [label="RAIN"]

    B -> D [label="HIGH"]
    B -> F [label="NORMAL"]

    C -> F [label="WEAK"]
    C -> G [label="STRONG"]
}
"""

s = Source(dot_code, filename="decision_tree", format="png")
s.view()

'decision_tree.png'